In [1]:
#!pip install sentence-transformers faiss-cpu openai numpy pymupdf

In [2]:
import sentence_transformers, transformers, huggingface_hub
print("sentence-transformers:", sentence_transformers.__version__)
print("transformers:", transformers.__version__)
print("huggingface_hub:", huggingface_hub.__version__)

sentence-transformers: 2.2.2
transformers: 4.31.0
huggingface_hub: 0.14.1


In [3]:
#!git clone https://gitlab.dkrz.de/icon/icon-model.git documents

In [4]:
import numpy as np
import os
import fitz  # PyMuPDF for PDF text extraction

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Load documents from a folder recursively
def load_documents_from_folder(folder_path):
    documents = []
    file_info = []
    
    for dirpath, dirnames, filenames in os.walk(folder_path):
        # Exclude .git directories
        dirnames[:] = [d for d in dirnames if not d.startswith('.git')]

        for filename in filenames:
            if filename.startswith('.git'):
                continue  # Skip any .git files

            file_path = os.path.join(dirpath, filename)
            if filename.endswith(('.f90','.txt', '.org', '.sh', '.toml','.pdf')) \
                or '.' not in filename:  # Load .txt, .sh, .pdf, and files without extensions
                try:
                    if filename.endswith('.pdf'):
                        content = extract_text_from_pdf(file_path)
                    else:
                        with open(file_path, 'r', encoding='utf-8') as file:
                            content = file.read()
                    documents.append(content)
                    file_info.append((filename, file_path))
                    print(f"Added to DB: {file_path}")
                except Exception as e:
                    print(f"Failed to process {file_path}: {e}")
    
    return documents, file_info

In [5]:
folder_path = './documents'  # Replace with your folder path
documents, file_info = load_documents_from_folder(folder_path)

Added to DB: ./documents/make_runscripts
Added to DB: ./documents/REUSE.toml
Added to DB: ./documents/data/README
Added to DB: ./documents/doc/ICON-PrivacyPolicy.pdf
Added to DB: ./documents/doc/Namelist_overview.pdf
Added to DB: ./documents/doc/config/icon-config-doc-depgraph.sh
Added to DB: ./documents/doc/contrib/nproma_1.pdf
Added to DB: ./documents/doc/contrib/nproma_3d.pdf
Added to DB: ./documents/doc/science/atm_echam_phy/icon_atm_echam_phy_scidoc.pdf
Added to DB: ./documents/doc/science/atm_echam_phy/Makefile
Added to DB: ./documents/doc/science/atm_echam_phy/make_icon_guide.sh
Added to DB: ./documents/doc/science/dynamics/dwdlogo.pdf
Added to DB: ./documents/doc/science/dynamics/fig_edge_volume.pdf
Added to DB: ./documents/doc/science/dynamics/fig_main_boxes.pdf
Added to DB: ./documents/doc/science/dynamics/fig_primal_dual.pdf
Added to DB: ./documents/doc/science/dynamics/fig_vortex_boxes.pdf
Added to DB: ./documents/doc/science/dynamics/icon_science.pdf
Added to DB: ./documen

In [6]:
print("Found", len(documents), "documents")

Found 2391 documents


In [7]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [8]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Load documents and create embeddings
documents, file_info = load_documents_from_folder("documents/")
embeddings = embedder.encode(documents, convert_to_numpy=True)

# Sanity check
print("✅ Loaded documents:", len(documents))
print("🔢 Embedding shape:", embeddings.shape)
print("📄 Example file:", file_info[0])
print("📄 Example snippet:\n", documents[0][:200])

Added to DB: documents/make_runscripts
Added to DB: documents/REUSE.toml
Added to DB: documents/data/README
Added to DB: documents/doc/ICON-PrivacyPolicy.pdf
Added to DB: documents/doc/Namelist_overview.pdf
Added to DB: documents/doc/config/icon-config-doc-depgraph.sh
Added to DB: documents/doc/contrib/nproma_1.pdf
Added to DB: documents/doc/contrib/nproma_3d.pdf
Added to DB: documents/doc/science/atm_echam_phy/icon_atm_echam_phy_scidoc.pdf
Added to DB: documents/doc/science/atm_echam_phy/Makefile
Added to DB: documents/doc/science/atm_echam_phy/make_icon_guide.sh
Added to DB: documents/doc/science/dynamics/dwdlogo.pdf
Added to DB: documents/doc/science/dynamics/fig_edge_volume.pdf
Added to DB: documents/doc/science/dynamics/fig_main_boxes.pdf
Added to DB: documents/doc/science/dynamics/fig_primal_dual.pdf
Added to DB: documents/doc/science/dynamics/fig_vortex_boxes.pdf
Added to DB: documents/doc/science/dynamics/icon_science.pdf
Added to DB: documents/doc/science/dynamics/mpilogo.pdf


In [9]:
print("type: ", type(embeddings))
print("shape: ", embeddings.shape)
print("MB: ", embeddings.nbytes / (1024**2))

type:  <class 'numpy.ndarray'>
shape:  (2391, 384)
MB:  3.50244140625


In [10]:
import faiss

# Create FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

In [11]:
# Query function
def query_vector_db(query, k=2):
    """
    Searches the FAISS index for the k most similar embeddings to the query.

    Parameters:
    query (str): Input text to search for similar documents.
    k (int): Number of nearest neighbors to retrieve.

    Returns:
    list of tuples: Each tuple contains (retrieved_document, file_info).
    """
    query_embedding = get_embedding(query).astype(np.float32)  # Convert to FAISS format
    distances, indices = index.search(query_embedding, k)  # Perform similarity search
    return [(documents[idx], file_info[idx]) for idx in indices[0]]

In [12]:
def query_vector_db(query, k=5):
    # Embed the query
    query_embedding = embedder.encode([query], convert_to_numpy=True)

    # Search FAISS index
    distances, indices = index.search(query_embedding, k)
    
    # Return matched documents and metadata
    results = []
    for i in indices[0]:
        doc = documents[i]
        info = file_info[i] if i < len(file_info) else "No info"
        results.append((doc, info))
    
    return results


In [13]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [14]:
# Step 6: Use OpenAI to discuss results
def chat_with_openai(query,k=12):
    retrieved_docs = query_vector_db(query,k=k)
    context = "\n\n".join([f"File: {file[0]}\nContent: {doc}" for doc, file in retrieved_docs])
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # using the gpt-4o-mini model
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Based on the following documents, answer the question:\n{context}\n\nQuestion: {query}"}
        ]
    )
    
    answer = response.choices[0].message.content
    
    # Append file references to the answer
    file_references = "\n\nReferenced Files:\n" + "\n".join([f"{file[0]}: {file[1]}" for _, file in retrieved_docs])
    
    return answer + file_references

In [15]:
# OpenAI Answers
query = "Tell me about the structure of the ICON model"
answer = chat_with_openai(query)

from IPython.display import display, Markdown
display(Markdown(answer))

The ICON (ICOsahedral Nonhydrostatic) model is a flexible and advanced numerical weather prediction model that utilizes a unique structure for simulating atmospheric processes. Its design is made to facilitate high-performance computing and to effectively run simulations on a variety of spatial scales. Below are the key components and structure of the ICON model:

### 1. **Modular Design**
- **Components Modules**: ICON is designed with a modular approach, allowing different physical processes and dynamical systems to be configured as separate modules. Each module can be integrated or modified without impacting the rest of the model significantly.
- **Submodules and Interfaces**: Each component has defined interfaces, which handle the interactions between modules, such as exchanging data and calling necessary functions. Examples include `mo_art_init_interface`, `mo_ocean_diagnostics`, and others.

### 2. **Numerical Grid**
- **Icosahedral Grid**: The ICON model employs a quasi-uniform grid based on the icosahedron, allowing for high-resolution simulations without significant distortion of geographical features.
- **Variable Resolution**: The grid can be adapted, allowing for high resolution in areas of interest (like regions of storm activity) while using coarser resolution in less critical areas.

### 3. **Physical Processes**
- **Microphysics**: The model includes modules for cloud microphysics (`mo_art_2mom_prepare`), which deals with processes like condensation and precipitation.
- **Dynamics**: The model implements advanced dynamical systems that consider the interactions between different atmospheric layers.
- **Ocean and Ice Processes**: Special modules exist for oceanic processes and sea-ice dynamics (`mo_ocean_check_total_content`, `mo_ice_fem_icon_init`), which handle interactions between the ocean and atmosphere.

### 4. **Multi-Scale Capability**
- **Nestings of Domains**: ICON has capabilities to work with nested domains, allowing for high-resolution simulations in selected areas while still modeling the larger scale dynamics.
- **Coupling with Other Models**: The model can interact with various earth system components (e.g., terrestrial, oceanic) through interfaces, enhancing its use as a part of larger coupled models.

### 5. **Data Handling**
- **Data Structures**: ICON uses complex data structures to manage variables across different horizontal, vertical, and temporal resolutions. This includes arrays for atmospheric states, tracer fields, and diagnostic variables.
- **Input/Output Management**: The model has systems in place for reading from and writing to files, ensuring efficient handling of large datasets required for simulations and diagnostics.

### 6. **Initialization and Configuration**
- **Parameterization**: Before running simulations, ICON requires a series of initialization routines that set physical parameters and configure necessary variables (`mo_art_init_interface`).
- **Dynamic Configuration**: The model's behavior can be adjusted dynamically based on user-defined configurations or run-time parameters.

### 7. **Performance Optimization**
- **OpenMP Optimization**: The model is designed to take advantage of parallel computing resources using OpenMP, enabling it to run efficiently on multi-core processors.
- **GPU Acceleration**: There are provisions for leveraging GPU resources to perform calculations, which can significantly enhance performance.

### 8. **Diagnostics and Monitoring**
- **Diagnostic Outputs**: Throughout the simulation, various diagnostics can be calculated and monitored, which help in analyzing the performance and accuracy of the model outputs.
- **Error and Volume Checks**: The model includes subsystems for verifying the conservation of mass and energy, as well as tracking accumulated errors.

In summary, the ICON model is a sophisticated framework that integrates several processes including atmospheric dynamics, ocean interactions, and computational optimizations to provide a versatile tool for weather prediction and climate modeling. Its modular architecture allows customizable configurations, making it suitable for a wide range of applications in meteorology and climate science.

Referenced Files:
AUTHORS.txt: documents/externals/fortran-support/AUTHORS.txt
mo_art_2mom_prepare.f90: documents/externals/art/phy_interact/mo_art_2mom_prepare.f90
AUTHORS.txt: documents/externals/math-support/AUTHORS.txt
AUTHORS.txt: documents/externals/math-interpolation/AUTHORS.txt
mo_art_general_interface.f90: documents/externals/art/interface/mo_art_general_interface.f90
mo_art_init_interface.f90: documents/externals/art/interface/mo_art_init_interface.f90
mo_test_jitter.f90: documents/src/testbed/mo_test_jitter.f90
mo_art_coagulation_interface.f90: documents/externals/art/interface/mo_art_coagulation_interface.f90
mo_art_external_init_pollen.f90: documents/externals/art/shared/mo_art_external_init_pollen.f90
mo_ocean_check_total_content.f90: documents/src/ocean/diagnostics/mo_ocean_check_total_content.f90
mo_ocean_diagnostics.f90: documents/src/ocean/diagnostics/mo_ocean_diagnostics.f90
mo_ice_fem_icon_init.f90: documents/src/sea_ice/dynamics_fem/mo_ice_fem_icon_init.f90

In [16]:
import ollama

def chat_with_ollama(query, model="mistral"):
    """Queries Ollama's local LLM (Mistral or DeepSeek) with retrieved VectorDB documents."""
    retrieved_docs = query_vector_db(query, k=20)
    context = "\n\n".join([f"File: {file[0]}\nContent: {doc}" for doc, file in retrieved_docs])

    full_prompt = f"Based on the following documents, answer the question:\n{context}\n\nQuestion: {query}"

    response = ollama.chat(model=model, messages=[{"role": "user", "content": full_prompt}])

    answer = response['message']['content']

    # Append file references
    file_references = "\n\nReferenced Files:\n" + "\n".join([f"{file[0]}: {file[1]}" for _, file in retrieved_docs])

    return answer + file_references


In [17]:
query_text = "Tell me about the structure of the ICON model"
answer = chat_with_ollama(query_text, model="deepseek-r1:1.5b")

# Display response as formatted Markdown
display(Markdown(answer))

<think>
Alright, I need to figure out how to answer the question about the structure of the ICON model. Let's start by recalling what I know from the provided code snippets and context.

First, I see that the user has included some F90 files related to the ICON model. The file mo_art_wrapper_routines.f90 is a module that uses several other modules like mo_loopindices, mo_impl_constants, etc., which suggests it's part of the model's data management or routine handling.

I notice that in the F90 code, there's a use statement for mo_loopindices and mo_impl_constants. These might be related to how arrays are handled across different host models, especially when using multiple platforms like DWD, MPI-M, DKRZ, KIT, and MeteoSwiss.

Looking at the code, I see that in mo_art_wrapper_routines.f90, there's a SUBROUTINE art_get_indices_c which uses get_indices_c from mo_loopindices. This function probably retrieves indices or parameters for certain models. The argument p_patch(jg) suggests it might be using patch numbers to reference different parts of the model.

I also see that in the F90 file, variables are used like TYPE(t_art_atmo), which implies that t_art_atmo is a type alias for some other structure, possibly containing array definitions or parameters related to the ICON model. The use statements here likely define how arrays are passed and handled across different host models.

In the provided code snippets, I don't see functions from mo_art_data or mo_art_atmo_data. However, in the F90 file, there's a mention of art_get_indices_c which is part of the wrapper for the original ICON routines. So, this function likely interacts with other parts that process arrays across different models.

Putting this together, I understand that the ICON model structure includes various components like patches, arrays for data storage, and functions that manage these arrays across multiple host models. The use statements in mo_art_wrapper_routines.f90 indicate how variables are defined and passed between different modules, probably using type aliases to define array structures.

I also note the mention of get_indices_c, which seems to retrieve indices based on parameters like block starting and ending, along with patch numbers. This could be crucial for determining where data is stored or processed in the model.

In summary, to understand the structure of the ICON model, I need to focus on how it handles patches, arrays, and how these are accessed across different host models using functions like art_get_indices_c. The F90 code provides specific implementations that use other modules for array definitions, which helps in tracking how data is managed within the model.
</think>

The ICON (Icosahedral Nonhydrostatic) model is a numerical weather prediction system that uses an icosahedral grid to represent atmospheric features. Its structure includes several key components:

1. **Grid Cells and Patches**: The model divides the Earth's surface into patches, typically around the poles as they are stretched in spherical coordinates.

2. **Variable Names**:
   - `t_art_atmo` likely refers to arrays defined for the ICON model.
   - Other variables are placeholders for data structures.

3. **Array Definitions**:
   ```fortran
   TYPE(t_art_atmo), POINTER :: art_atmo
   ```
   This defines a variable `art_atmo` as a type alias with a pointer, used to hold arrays relevant to the ICON model.

4. **Functionality**:
   - The `get_indices_c` function in `mo_art_wrapper_routines.f90` retrieves indices based on block parameters and patch numbers.
   - This suggests processing of data structures for various models.

5. **Host Model Compatibility**: The use of other modules (like mo_loopindices and mo_impl_constants) indicates handling array definitions across multiple host models, crucial for platform-specific operations.

In summary, the ICON model's structure is defined by its grid cells, patches, variable names, and array handling, all managed using F90 code that interacts with other modules for data processing.

Referenced Files:
AUTHORS.txt: documents/externals/fortran-support/AUTHORS.txt
mo_art_2mom_prepare.f90: documents/externals/art/phy_interact/mo_art_2mom_prepare.f90
AUTHORS.txt: documents/externals/math-support/AUTHORS.txt
AUTHORS.txt: documents/externals/math-interpolation/AUTHORS.txt
mo_art_general_interface.f90: documents/externals/art/interface/mo_art_general_interface.f90
mo_art_init_interface.f90: documents/externals/art/interface/mo_art_init_interface.f90
mo_test_jitter.f90: documents/src/testbed/mo_test_jitter.f90
mo_art_coagulation_interface.f90: documents/externals/art/interface/mo_art_coagulation_interface.f90
mo_art_external_init_pollen.f90: documents/externals/art/shared/mo_art_external_init_pollen.f90
mo_ocean_check_total_content.f90: documents/src/ocean/diagnostics/mo_ocean_check_total_content.f90
mo_ocean_diagnostics.f90: documents/src/ocean/diagnostics/mo_ocean_diagnostics.f90
mo_ice_fem_icon_init.f90: documents/src/sea_ice/dynamics_fem/mo_ice_fem_icon_init.f90
mo_art_fplume_types.f90: documents/externals/art/externals/fplume/mo_art_fplume_types.f90
mo_art_pntSrc_state.f90: documents/externals/art/shared/mo_art_pntSrc_state.f90
mo_art_2mom_main.f90: documents/externals/art/phy_interact/mo_art_2mom_main.f90
mo_art_aerodyn_interface.f90: documents/externals/art/interface/mo_art_aerodyn_interface.f90
mo_art_oem_types.f90: documents/externals/art/emissions/mo_art_oem_types.f90
mo_art_init_all_dom.f90: documents/externals/art/shared/mo_art_init_all_dom.f90
mo_art_read_pol_oper.f90: documents/externals/art/io/mo_art_read_pol_oper.f90
mo_art_wrapper_routines.f90: documents/externals/art/shared/mo_art_wrapper_routines.f90

In [18]:
answer = chat_with_ollama(query_text, model="mistral")

# Display response as formatted Markdown
display(Markdown(answer))

 The ICON (ICON-LEPS, Integrated Forecasting System with Global and Regional Ensemble Prediction System) model is a numerical weather prediction model developed by the German Weather Service (Deutscher Wetterdienst, DWD). It's used for operational medium-range to seasonal predictions.

   The provided code snippet is part of the ICON model, specifically within its Fortran modules. The code demonstrates how data from the Artificial Intelligence-based Coupled global climate Model (ART) is handled and integrated with the ICON model for different host models or uses as a standalone model.

   The `mo_art_read_pol_oper` subroutine reads external polar grid data using ART's input variables defined in the `vsuff` array. It then processes these fields, depending on their definitions (input field is not). The read data is stored in the `this_pollen_table`. Finally, it deallocates the memory used by the `vsuff` array.

   The `mo_art_wrapper_routines` module collects various routines that use ICON information and are essential for another host model or need to be adapted for other host models or using ART as a standalone model. The code snippet provided shows the `art_get_indices_c` subroutine, which is used to get indices for the ICON grid from a given patch (a set of adjacent grid cells).

   This structure demonstrates how complex weather prediction models like ICON are designed with multiple components, each performing specific functions, and how they integrate with other systems like ART to increase their predictive capabilities.

Referenced Files:
AUTHORS.txt: documents/externals/fortran-support/AUTHORS.txt
mo_art_2mom_prepare.f90: documents/externals/art/phy_interact/mo_art_2mom_prepare.f90
AUTHORS.txt: documents/externals/math-support/AUTHORS.txt
AUTHORS.txt: documents/externals/math-interpolation/AUTHORS.txt
mo_art_general_interface.f90: documents/externals/art/interface/mo_art_general_interface.f90
mo_art_init_interface.f90: documents/externals/art/interface/mo_art_init_interface.f90
mo_test_jitter.f90: documents/src/testbed/mo_test_jitter.f90
mo_art_coagulation_interface.f90: documents/externals/art/interface/mo_art_coagulation_interface.f90
mo_art_external_init_pollen.f90: documents/externals/art/shared/mo_art_external_init_pollen.f90
mo_ocean_check_total_content.f90: documents/src/ocean/diagnostics/mo_ocean_check_total_content.f90
mo_ocean_diagnostics.f90: documents/src/ocean/diagnostics/mo_ocean_diagnostics.f90
mo_ice_fem_icon_init.f90: documents/src/sea_ice/dynamics_fem/mo_ice_fem_icon_init.f90
mo_art_fplume_types.f90: documents/externals/art/externals/fplume/mo_art_fplume_types.f90
mo_art_pntSrc_state.f90: documents/externals/art/shared/mo_art_pntSrc_state.f90
mo_art_2mom_main.f90: documents/externals/art/phy_interact/mo_art_2mom_main.f90
mo_art_aerodyn_interface.f90: documents/externals/art/interface/mo_art_aerodyn_interface.f90
mo_art_oem_types.f90: documents/externals/art/emissions/mo_art_oem_types.f90
mo_art_init_all_dom.f90: documents/externals/art/shared/mo_art_init_all_dom.f90
mo_art_read_pol_oper.f90: documents/externals/art/io/mo_art_read_pol_oper.f90
mo_art_wrapper_routines.f90: documents/externals/art/shared/mo_art_wrapper_routines.f90

In [19]:
import os
import shutil

def backup_and_create_results_folder(base_folder="results"):
    """Backs up existing results folder by renaming it to results_nnn and creates a new empty folder."""
    
    if os.path.exists(base_folder):
        counter = 1
        while os.path.exists(f"{base_folder}_{counter:03d}"):
            counter += 1
        backup_folder = f"{base_folder}_{counter:03d}"
        shutil.move(base_folder, backup_folder)
        print(f"Existing results folder backed up as: {backup_folder}")

    os.makedirs(base_folder, exist_ok=True)
    return base_folder

def copy_retrieved_documents(query, k=10, results_folder="results"):
    """Finds relevant documents using FAISS and copies them to the results folder, saving the query."""
    
    # Backup old results and create new folder
    results_folder = backup_and_create_results_folder(results_folder)

    # Retrieve relevant documents
    retrieved_docs = query_vector_db(query, k)
    
    copied_files = []
    
    for _, file_info in retrieved_docs:
        file_path = file_info[1]  # Assuming file_info[1] contains the file path

        if os.path.exists(file_path):
            dest_path = os.path.join(results_folder, os.path.basename(file_path))
            shutil.copy(file_path, dest_path)
            copied_files.append(dest_path)
        else:
            print(f"Warning: File not found - {file_path}")

    # Save query text
    query_path = os.path.join(results_folder, "query.txt")
    with open(query_path, "w", encoding="utf-8") as f:
        f.write(query)
    
    print(f"Copied {len(copied_files)} files to {results_folder}")
    print(f"Query saved in {query_path}")


In [20]:
# Example usage
query_text = "What turbulence scheme in ICON?"
copy_retrieved_documents(query_text, k=5)

Existing results folder backed up as: results_001
Copied 5 files to results
Query saved in results/query.txt


In [21]:
# OpenAI Answers
query = "What turbulence scheme in ICON?"
answer = chat_with_openai(query)

from IPython.display import display, Markdown
display(Markdown(answer))

The turbulence scheme in ICON refers to a set of computational methods used for modeling atmospheric turbulence, specifically focused on the interaction between the atmosphere and the land surface. In the reference files you provided, two primary turbulence schemes are mentioned: the "COSMO turbulence scheme" developed by Matthias Raschendorfer and "VDIFF" (Vertical Diffusion) turbulence scheme.

### Key Components of the Turbulence Scheme in ICON:

1. **COSMO Turbulence Scheme**:
   - This scheme is implemented as part of the interface to ART (Aerosol-Related Turbulent) routines for turbulent diffusion, as detailed in `mo_art_turbdiff_interface.f90`.
   - The implementation provides a way to handle turbulent diffusion of passive tracers using the COSMO turbulence model.

2. **VDIFF Turbulence Scheme**:
   - The VDIFF scheme is described in `mo_turb_vdiff_config.f90`, `mo_turb_vdiff_params.f90`, and various other files.
   - It involves the use of a configuration structure (type `t_vdiff_config`), which includes parameters for surface momentum and heat flux, turbulent Prandtl numbers, and other constants relevant for the turbulence model.
   - This scheme is particularly focused on the eddy diffusivity mass flux method for representing turbulence within the atmospheric boundary layer.

3. **Subgrid-Scale Turbulence**:
   - Another aspect highlighted is the `les_turbulence` subroutine which manages subgrid turbulence scaling and interaction with the overall atmospheric modeling framework.
   - The use of momentum and scalar variables is key for capturing the dynamics of turbulent flows.

4. **Turbulence Features**:
   - There are various subroutines for initializing turbulent parameters, adjusting saturation, solving turbulent budgets, and tracking different variables related to turbulence.
   - The algorithms incorporate aspects ranging from direct numerical simulation (DNS) and large eddy simulation (LES) to more parameterized approaches suitable for operational weather forecasting.

5. **Turbulent Diagnostics**:
   - The model also includes capabilities for calculating and writing out turbulent diagnostics (mean fields, variances, etc.) which are crucial for evaluating the turbulence scheme's performance.

6. **Use of OpenACC**:
   - The code uses OpenACC directives for parallelization, allowing the model to leverage GPU resources for computational efficiency.

### Summary
In summary, the ICON model's turbulence scheme includes the COSMO turbulence scheme for turbulent diffusion and the VDIFF method for vertical diffusion, among others. These schemes are designed to handle the complexities of atmospheric turbulence and the interactions of different variables in a numerically efficient manner.

Referenced Files:
mo_art_turbdiff_interface.f90: documents/externals/art/interface/mo_art_turbdiff_interface.f90
mo_turb_vdiff_config.f90: documents/src/configure_model/mo_turb_vdiff_config.f90
mo_turb_vdiff_params.f90: documents/src/atm_phy_schemes/mo_turb_vdiff_params.f90
mo_turb_config_class.f90: documents/externals/jsbach/src/turbulence/mo_turb_config_class.f90
mo_interface_aes_tmx.f90: documents/src/atm_phy_aes/mo_interface_aes_tmx.f90
turb_utilities.f90: documents/src/atm_phy_schemes/turb_utilities.f90
turb_data.f90: documents/src/atm_phy_schemes/turb_data.f90
data_turbulence.f90: documents/src/atm_phy_schemes/data_turbulence.f90
mo_turbulent_diagnostic.f90: documents/src/atm_phy_les/mo_turbulent_diagnostic.f90
mo_tmx_smagorinsky.f90: documents/src/atm_phy_aes/tmx/mo_tmx_smagorinsky.f90
mo_les_turb_interface.f90: documents/src/atm_phy_les/mo_les_turb_interface.f90
mo_nwp_turbdiff_interface.f90: documents/src/atm_phy_nwp/mo_nwp_turbdiff_interface.f90

In [22]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
%ls

ai_tutorial.pdf*  documents/  results/  results_001/  vectordb.ipynb*


In [23]:
from PyPDF2 import PdfReader

def extract_pages_from_pdf(pdf_path):
    """Reads a PDF and returns a list of page texts."""
    reader = PdfReader(pdf_path)
    return [page.extract_text() for page in reader.pages]

In [24]:
# Load the ICON tutorial as separate pages
tutorial_pdf = "ai_tutorial.pdf"
tutorial_pages = extract_pages_from_pdf(tutorial_pdf)

print(f"✅ Extracted {len(tutorial_pages)} pages from {tutorial_pdf}")

# Generate references like "ai_tutorial.pdf - Page 1", ...
tutorial_page_refs = [f"{tutorial_pdf} - Page {i+1}" for i in range(len(tutorial_pages))]

# Add to main document and file info lists
documents.extend(tutorial_pages)
file_info.extend([(ref, tutorial_pdf) for ref in tutorial_page_refs])

✅ Extracted 196 pages from ai_tutorial.pdf


In [25]:
tutorial_embeddings = embedder.encode(tutorial_pages, convert_to_numpy=True)

In [26]:
print("FAISS Index Size:", index.ntotal)  # Total embeddings in FAISS
print("Documents list size:", len(documents))
print("File Info size:", len(file_info))

FAISS Index Size: 2391
Documents list size: 2587
File Info size: 2587


In [27]:
index.add(tutorial_embeddings)

In [28]:
print("FAISS Index Size:", index.ntotal)  # Total embeddings in FAISS
print("Documents list size:", len(documents))
print("File Info size:", len(file_info))

FAISS Index Size: 2587
Documents list size: 2587
File Info size: 2587


In [29]:
# OpenAI Answers
query = "What NN architectures did we talk about? CNN? GNN? Transformers?"
answer = chat_with_openai(query, k=20)

from IPython.display import display, Markdown
display(Markdown(answer))

Based on the documents provided, the discussed neural network (NN) architectures include:

1. **Feedforward Neural Networks (FFNN)** - The simplest form of artificial neural networks where information moves in one direction from the input layer through hidden layers to the output layer.

2. **Convolutional Neural Networks (CNN)** - Powerful architectures typically used for image processing and can be applied to one-dimensional data, such as time series and function classification.

3. **Recurrent Neural Networks (RNN)** - Particularly highlighted for sequence learning, with types like Long Short-Term Memory (LSTM) networks, which improved the processing of language sequences.

4. **Transformers** - Introduced groundbreaking innovations with self-attention mechanisms and parallel processing, becoming the standard for modern language models.

The documents specifically mention CNNs and Transformers, while RNNs (including LSTMs) are noted for their advancements in sequence processing. There was no mention of Graph Neural Networks (GNN) in the provided materials.

Referenced Files:
ai_tutorial.pdf - Page 5: ai_tutorial.pdf
ai_tutorial.pdf - Page 188: ai_tutorial.pdf
ai_tutorial.pdf - Page 111: ai_tutorial.pdf
ai_tutorial.pdf - Page 182: ai_tutorial.pdf
ai_tutorial.pdf - Page 101: ai_tutorial.pdf
ai_tutorial.pdf - Page 90: ai_tutorial.pdf
ai_tutorial.pdf - Page 124: ai_tutorial.pdf
ai_tutorial.pdf - Page 130: ai_tutorial.pdf
ai_tutorial.pdf - Page 113: ai_tutorial.pdf
ai_tutorial.pdf - Page 192: ai_tutorial.pdf
ai_tutorial.pdf - Page 190: ai_tutorial.pdf
ai_tutorial.pdf - Page 102: ai_tutorial.pdf
ai_tutorial.pdf - Page 191: ai_tutorial.pdf
ai_tutorial.pdf - Page 87: ai_tutorial.pdf
ai_tutorial.pdf - Page 120: ai_tutorial.pdf
ai_tutorial.pdf - Page 131: ai_tutorial.pdf
ai_tutorial.pdf - Page 114: ai_tutorial.pdf
ai_tutorial.pdf - Page 193: ai_tutorial.pdf
ai_tutorial.pdf - Page 126: ai_tutorial.pdf
ai_tutorial.pdf - Page 167: ai_tutorial.pdf

In [30]:
answer = chat_with_ollama(query, model="deepseek-r1:1.5b")

# Display response as formatted Markdown
display(Markdown(answer))

<think>
Okay, so I'm trying to figure out what types of neural networks were discussed in the provided text. Let me start by reading through each part carefully and noting down any terms or concepts that stand out as neural network (NN) architectures.

First, in Chapter 5, there's a section titled "Transformers" where they talk about the self-attention mechanism. Transformers themselves are known for their attention mechanisms, which use multi-head attention and positional encoding. So I can identify transformers as one type of NN architecture mentioned.

Looking at Chapter 6, under "Large Language Models," the text discusses Transformer architectures in detail. They explain how each token is embedded into a vector using an embedding matrix and then processed with self-attention. This reinforces that Transformers are central to the examples provided.

In Chapter 9, titled "Multimodal LLMs," the focus shifts to models combining different data modalities like text, images, etc. The part about wind field generation uses T5 for fine-tuning a Transformer model. This again points towards Transformers being used in various contexts beyond traditional LLMs.

Additionally, looking at Chapter 12 under "Large Language Models" has content on CNNs, but the user's question specifically asks about NN architectures, so I should focus there. The text discusses CNNs (which are Convolutional Neural Networks), focusing on their architecture, activation functions, pooling layers, and common applications like image classification.

Wait, in Chapter 12 under "Large Language Models," it mentions CNNs but the main discussion is about multilayer perceptrons for convolutional layers. So perhaps the main NN architectures here are Multilayer Perceptrons (MLPs) and their connection to CNNs through feature extraction. This seems more relevant since the question is about NN architectures beyond Transformers.

Putting it all together, the main NN architectures discussed include:
1. Transformers: Mentioned in Chapter 5 and 6.
2. CNNs/MLPs: Focused on in Chapter 12, discussing their architecture, functions, and applications.

So I think that's it. The key points are transformers for NLP tasks, and CNNs (including MLPs) for image-related tasks.
</think>

The main neural network architectures discussed in the provided text include:

1. **Transformers**: Used extensively, especially in chapters on large language models where they utilize self-attention mechanisms and positional encoding.

2. **CNNs/MLPs**: Discussed in a separate chapter focused on multilabel weather forecasting. CNNs are highlighted for their architecture, which includes convolutional layers and pooling, leading to the application of Multilayer Perceptrons (MLPs) for feature extraction.

Referenced Files:
ai_tutorial.pdf - Page 5: ai_tutorial.pdf
ai_tutorial.pdf - Page 188: ai_tutorial.pdf
ai_tutorial.pdf - Page 111: ai_tutorial.pdf
ai_tutorial.pdf - Page 182: ai_tutorial.pdf
ai_tutorial.pdf - Page 101: ai_tutorial.pdf
ai_tutorial.pdf - Page 90: ai_tutorial.pdf
ai_tutorial.pdf - Page 124: ai_tutorial.pdf
ai_tutorial.pdf - Page 130: ai_tutorial.pdf
ai_tutorial.pdf - Page 113: ai_tutorial.pdf
ai_tutorial.pdf - Page 192: ai_tutorial.pdf
ai_tutorial.pdf - Page 190: ai_tutorial.pdf
ai_tutorial.pdf - Page 102: ai_tutorial.pdf
ai_tutorial.pdf - Page 191: ai_tutorial.pdf
ai_tutorial.pdf - Page 87: ai_tutorial.pdf
ai_tutorial.pdf - Page 120: ai_tutorial.pdf
ai_tutorial.pdf - Page 131: ai_tutorial.pdf
ai_tutorial.pdf - Page 114: ai_tutorial.pdf
ai_tutorial.pdf - Page 193: ai_tutorial.pdf
ai_tutorial.pdf - Page 126: ai_tutorial.pdf
ai_tutorial.pdf - Page 167: ai_tutorial.pdf

In [31]:
answer = chat_with_ollama(query, model="mistral")

# Display response as formatted Markdown
display(Markdown(answer))

 In this text, we talked about three types of neural network architectures: Convolutional Neural Networks (CNN), Graph Neural Networks (GNN), and Transformers. The CNN architecture was discussed in the context of image processing tasks, whereas Transformers were mentioned in relation to large language models and the self-attention mechanism. It appears that GNN wasn't directly addressed in this particular part of the text, but it could be inferred from other sections discussing graph-based problems like social network analysis or molecular simulations.

Referenced Files:
ai_tutorial.pdf - Page 5: ai_tutorial.pdf
ai_tutorial.pdf - Page 188: ai_tutorial.pdf
ai_tutorial.pdf - Page 111: ai_tutorial.pdf
ai_tutorial.pdf - Page 182: ai_tutorial.pdf
ai_tutorial.pdf - Page 101: ai_tutorial.pdf
ai_tutorial.pdf - Page 90: ai_tutorial.pdf
ai_tutorial.pdf - Page 124: ai_tutorial.pdf
ai_tutorial.pdf - Page 130: ai_tutorial.pdf
ai_tutorial.pdf - Page 113: ai_tutorial.pdf
ai_tutorial.pdf - Page 192: ai_tutorial.pdf
ai_tutorial.pdf - Page 190: ai_tutorial.pdf
ai_tutorial.pdf - Page 102: ai_tutorial.pdf
ai_tutorial.pdf - Page 191: ai_tutorial.pdf
ai_tutorial.pdf - Page 87: ai_tutorial.pdf
ai_tutorial.pdf - Page 120: ai_tutorial.pdf
ai_tutorial.pdf - Page 131: ai_tutorial.pdf
ai_tutorial.pdf - Page 114: ai_tutorial.pdf
ai_tutorial.pdf - Page 193: ai_tutorial.pdf
ai_tutorial.pdf - Page 126: ai_tutorial.pdf
ai_tutorial.pdf - Page 167: ai_tutorial.pdf